In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,20125
2,Huelva,Confirmados PDIA 14 días,5454
3,Huelva,Tasa PDIA 14 días,"1062,8056979168696"
4,Huelva,Confirmados PDIA 7 días,2518
5,Huelva,Total Confirmados,20350
6,Huelva,Curados,11169
7,Huelva,Fallecidos,179


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  20125.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  5373.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 238 personas en los últimos 7 días 

Un positivo PCR cada 117 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,20125.0,2518.0,5454.0,513170.0,490.675605,1062.805698,548.0
Huelva-Costa,12205.0,1529.0,3550.0,289548.0,528.064431,1226.048876,337.0
Condado-Campiña,5487.0,721.0,1334.0,156231.0,461.496118,853.863830,146.0
Huelva (capital),5373.0,604.0,1228.0,143837.0,419.919770,853.744169,130.0
Isla Cristina,1703.0,229.0,682.0,21393.0,1070.443603,3187.958678,59.0
Sierra de Huelva-Andévalo Central,2133.0,217.0,505.0,67391.0,322.001454,749.358223,42.0
Lepe,1602.0,191.0,428.0,27880.0,685.078910,1535.150646,40.0
Ayamonte,905.0,158.0,413.0,21104.0,748.673237,1956.974981,39.0
Bollullos Par del Condado,407.0,66.0,117.0,14387.0,458.747480,813.234170,24.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Villalba del Alcor,391.0,61.0,161.0,3366.0,1812.240048,4783.125371,6.0
Berrocal,14.0,6.0,13.0,303.0,1980.198020,4290.429043,0.0
Paymogo,54.0,18.0,40.0,1162.0,1549.053356,3442.340792,4.0
Isla Cristina,1703.0,229.0,682.0,21393.0,1070.443603,3187.958678,59.0
Encinasola,115.0,10.0,41.0,1310.0,763.358779,3129.770992,5.0
Calañas,153.0,14.0,77.0,2768.0,505.780347,2781.791908,2.0
Villanueva de las Cruces,17.0,2.0,10.0,387.0,516.795866,2583.979328,1.0
San Bartolomé de la Torre,146.0,39.0,80.0,3761.0,1036.958256,2127.093858,10.0
Ayamonte,905.0,158.0,413.0,21104.0,748.673237,1956.974981,39.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Minas de Riotinto,96.0,3.0,19.0,3812.0,78.698846,498.426023,1.0,0.157895
Puebla de Guzmán,65.0,3.0,18.0,3092.0,97.024580,582.147477,0.0,0.166667
Calañas,153.0,14.0,77.0,2768.0,505.780347,2781.791908,2.0,0.181818
Villanueva de las Cruces,17.0,2.0,10.0,387.0,516.795866,2583.979328,1.0,0.200000
Encinasola,115.0,10.0,41.0,1310.0,763.358779,3129.770992,5.0,0.243902
Cumbres Mayores,24.0,1.0,4.0,1749.0,57.175529,228.702115,0.0,0.250000
Cerro de Andévalo (El),120.0,12.0,45.0,2327.0,515.685432,1933.820370,3.0,0.266667
Aroche,81.0,2.0,6.0,3054.0,65.487885,196.463654,1.0,0.333333
Isla Cristina,1703.0,229.0,682.0,21393.0,1070.443603,3187.958678,59.0,0.335777
